In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
import numpy as np

In [15]:
movies = pd.read_csv("../data/movies.csv")
movies = movies.sort_values(by=['score'], ascending=False).head(10000)
movies

,Unnamed: 0,id,title,titleType,originalTitle,startYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes,score
50023,78380,tt0111161,The Shawshank Redemption,movie,The Shawshank Redemption,1994,142,Drama,nm0001104,nm0000175,9.3,2274619.0,9780861.7
80640,184735,tt0468569,The Dark Knight,movie,The Dark Knight,2008,152,Action,nm0634240,nm0634300,9.0,2236760.0,8947040.0
95371,263612,tt1375666,Inception,movie,Inception,2010,148,Action,nm0634240,nm0634240,8.8,1996195.0,7585541.0
49910,78182,tt0110912,Pulp Fiction,movie,Pulp Fiction,1994,154,Crime,nm0000233,nm0000233,8.9,1777018.0,6930370.2
49435,77420,tt0109830,Forrest Gump,movie,Forrest Gump,1994,142,Drama,nm0000709,nm0343165,8.8,1751980.0,6657524.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88033,218789,tt1050160,Kataude mashin gâru,movie,Kataude mashin gâru,2008,96,Action,nm1175724,nm1175724,5.9,9628.0,8665.2
36021,56309,tt0077864,Lo chiamavano Bulldozer,movie,Lo chiamavano Bulldozer,1978,110,Action,nm0526971,nm0104989,6.8,4814.0,8665.2
86116,207133,tt0976026,Cheeni Kum,movie,Cheeni Kum,2007,140,Comedy,nm2669564,nm2669564,6.8,4814.0,8665.2
14149,27245,tt0041239,Champion,movie,Champion,1949,99,Drama,nm0733476,nm0286025,7.4,3610.0,8664.0


In [16]:
mean_rating = movies['averageRating'].mean()

In [17]:
mean_rating

7.012339999999999

In [18]:
movies['error'] = mean_rating - movies['averageRating']

In [19]:
def rmse():
    out = 0
    for x in movies['error']:
        out += x**2

    return (out/movies['error'].size)**(0.5)

In [20]:
rmse()

0.7268794428239123

In [21]:
best_directors = movies.sort_values(by=['score', 'directors'], ascending=False).head(250)['directors'].unique().tolist()
best_writers = movies.sort_values(by=['score', 'writers'], ascending=False).head(250)['writers'].unique().tolist()

replacable_directors = []
replacable_writers = []


for x in movies['directors']:
    if x not in best_directors:
        replacable_directors.append(x)
        
for x in movies['writers']:
    if x not in best_directors:
        replacable_writers.append(x)       

In [23]:
print('# irreplacable directors : {}'.format(len(best_directors)))
print('# irreplacable writers : {}'.format(len(best_writers)))

# irreplacable directors : 154
# irreplacable writers : 188


In [25]:
movies.replace(replacable_directors, 'unknown', inplace=True)
movies.replace(replacable_writers, 'unknown', inplace=True)

In [29]:
#get dummies for: genres	directors	writers	
genre_dummies = pd.get_dummies(movies['genres'])
director_dummies = pd.get_dummies(movies['directors'])
writer_dummies = pd.get_dummies(movies['writers'])

frames = [movies['startYear'], genre_dummies, director_dummies, writer_dummies, movies['runtimeMinutes'], movies['score']]

In [32]:
ready_for_model = pd.concat(frames, axis=1)

In [33]:
ready_for_model

,startYear,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,...,nm1443502,nm1512910,nm1732981,nm2284484,nm2588606,nm3227090,nm3363032,unknown,runtimeMinutes,score
50023,1994,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,142,9780861.7
80640,2008,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,152,8947040.0
95371,2010,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,148,7585541.0
49910,1994,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,154,6930370.2
49435,1994,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,142,6657524.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88033,2008,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,96,8665.2
36021,1978,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,110,8665.2
86116,2007,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,140,8665.2
14149,1949,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,99,8664.0


In [ ]:
import sklearn.linear_model as sk

In [ ]:
model = sk.BayesianRidge()

In [ ]:
model

In [ ]:
movies['directors'].replace(lambda x: x not in best_directors, "unknown", inplace=True)